In [107]:
import ast, inspect

ALLOWED_FUNCTIONS = {'itertools', 'numpy', 'np', 'math'}
DISALLOWED_BUILTINS = {'print','__import__','breakpoint','compile','open','dir','eval','exec','globals','input','repr'}
ALLOWED_BUILTINS = {
    'abs', 'aiter', 'all', 'any', 'ascii', 'bin', 'bool', 'bytearray', 'bytes', 
    'callable', 'chr', 'classmethod', 'complex', 'delattr', 'dict', 
    'divmod', 'enumerate', 'filter', 'float', 'format', 'frozenset', 
    'getattr', 'hasattr', 'hash', 'help', 'hex', 'id', 'int', 
    'isinstance', 'issubclass', 'iter', 'len', 'list', 'locals', 'map', 
    'max', 'memoryview', 'min', 'next', 'object', 'oct', 'ord', 'pow', 
    'property', 'range', 'reversed', 'round', 'set', 'setattr', 'slice', 
    'sorted', 'staticmethod', 'str', 'sum', 'super', 'tuple', 'type', 
    'vars', 'zip'
}

class FunctionChecker(ast.NodeVisitor):
    def __init__(self):
        self.is_safe = True
        self.vars = []

    def visit_Import(self, node):
        for alias in node.names:
            if alias.name not in ALLOWED_FUNCTIONS:
                self.is_safe = False
            self.generic_visit(node)

    def visit_ImportFrom(self, node):
        # Now, keep track of which parts are allowed from the module
        if node.module not in ALLOWED_FUNCTIONS:
            self.is_safe = False  # Disallow the whole import if module isn't allowed
        for alias in node.names:
            if alias.name not in self.vars:
                self.vars.append(alias.name)  # Register imported names as safe variables
        self.generic_visit(node)
    
    def visit_Assign(self, node):
        for target in node.targets:
            # Check if the target is a name
            if isinstance(target, ast.Name):
                self.vars.append(target.id)
            # Check if the target is a tuple or list
            elif isinstance(target, (ast.Tuple, ast.List)):
                for element in target.elts:
                    if isinstance(element, ast.Name):
                        self.vars.append(element.id)

    def visit_arguments(self, node):
        for arg in node.args:
            if isinstance(arg, ast.arg):
                if arg.arg not in self.vars:
                    self.vars.append(arg.arg)
        self.generic_visit(node)

    def visit_Call(self, node):
        # # Check for disallowed built-in function calls
        if isinstance(node.func, ast.Name):
            print(node.func.id)
            if node.func.id in DISALLOWED_BUILTINS:
                self.is_safe = False
        # # Check if function calls are from allowed modules
        if isinstance(node.func, ast.Attribute):
            # if node.func.value.id not in ALLOWED_FUNCTIONS:
            #     self.is_safe = False
            func_value = ast.unparse(node.func.value)
            func_value = func_value.split('.')[0]
            if '(' in func_value:
                func_value = func_value.split('(')[0]
            if '[' in func_value:
                func_value = func_value.split('[')[0]
            if func_value not in ALLOWED_FUNCTIONS and func_value not in self.vars and func_value not in ALLOWED_BUILTINS:
                self.is_safe = False
        self.generic_visit(node)

def is_function_safe(func):
    function_code = inspect.getsource(func)
    tree = ast.parse(function_code)
    checker = FunctionChecker()
    checker.visit(tree)
    return checker.is_safe

def is_function_safe_string(function_code:str)->bool:
    #function_code = inspect.getsource(func)
    tree = ast.parse(function_code)
    checker = FunctionChecker()
    checker.visit(tree)
    return checker.is_safe

In [108]:
def priority_v2(v: tuple[int, ...], n: int) -> float:
    """Improves the priority calculation by considering the distribution of 0's, 1's, and 2's in the vector 'v'.
    The priority is inversely proportional to the number of occurrences of each unique value in 'v'.
    """
    unique_values = np.unique(v)
    counts = [list(v).count(val) for val in unique_values]
    if 1 in counts:
        priority = np.mean([1/c for c in counts if c != 1])
    else:
        priority = 1.0
    return priority

is_function_safe(priority_v2)

True

In [111]:
import numpy as np
import itertools

def my_function():
    from numpy import sqrt
    x = list([1,2,3,4])

    return sqrt(2)


print(is_function_safe(my_function))



sqrt
True


In [110]:
def priority1(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  pair_count = len(set(itertools.combinations(v, 2)))
  return pair_count / (n * (n - 1) / 2)


def priority2(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  v = sorted(v)
  pair_count = sum(1 for i in range(1, n) if v[i] != v[i-1])
  return pair_count / n


def priority3(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  unique_values = np.unique(v)
  pair_count = sum(1 for i in range(n) for j in range(i+1, n) if v[i] != v[j])
  diversity = len(unique_values) / n
  return pair_count / n + diversity


def priority4(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  unique_elements = len(set(v))
  pair_count = sum(1 for i in range(n) for j in range(i+1, n) if v[i] != v[j])
  return (unique_elements * pair_count) / (n * (n-1))

def priority5(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  # This function is identical to `priority_v2`; we're only adding it to maintain the sequence of function names
  return priority_v2(v, n)


def priority6(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  count = 0
  for i in range(n):
    for j in range(i+1, n):
      if v[i] != v[j]:
        count += 1
  return count / n


def priority7(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  # Convert vector to a bitmask by inverting each 3-valued entry and joining them as a single binary number
  bitmask = sum(2**i for i in range(n) if v[i] != 0)
  # Calculate the number of 1s in the bitmask to get the pair count
  pair_count = np.popcount(bitmask)
  return pair_count / n


def priority8(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  unique_pairs = len(set(list(itertools.combinations(v, 2))))
  return unique_pairs / np.math.comb(n, 2)

def priority9(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  unique_pairs = open('test')
  return unique_pairs / np.math.comb(n, 2)

def priority10(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
      The cap set will be constructed by adding vectors that do not create a line in order by priority.
      This version checks for collinearity by calculating the determinant of the 3x3 matrix made from the vectors.
  """
  # Calculate the dot product of each pair of vectors
  dot_products = np.outer(v, v)[::-1].sum(axis=0)

  # Add a small epsilon to avoid singularity
  epsilon = 1e-8
  determinant = np.linalg.det(np.vstack((v, dot_products, np.ones(n))).T + epsilon * np.eye(n + 1))

  # Normalize the determinant to a range of [0, 1]
  return 1.0 - np.abs(determinant) / (np.linalg.det(np.eye(n + 1)) ** 2)

def priority11(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
      The cap set will be constructed by adding vectors that do not create a line in order by priority.
      This version uses a simple heuristic to prioritize vectors with fewer lines.
  """
  lines = set()
  for i in range(n):
    for j in range(i+1, n):
      if np.sum(np.array(v[i:j+1]) == np.array(v[i:j+1]).sum(axis=0)) == 3:
        lines.add((i, j))
  if lines:
    return len(lines)
  else:
    return 0.0

In [105]:
# %timeit is_function_safe_string(inspect.getsource(priority1))
# %timeit is_function_safe_string(inspect.getsource(priority2))
# %timeit is_function_safe_string(inspect.getsource(priority3))
# %timeit is_function_safe_string(inspect.getsource(priority4))
# %timeit is_function_safe_string(inspect.getsource(priority5))
# %timeit is_function_safe_string(inspect.getsource(priority6))
# %timeit is_function_safe_string(inspect.getsource(priority7))
# %timeit is_function_safe_string(inspect.getsource(priority8))
# %timeit is_function_safe_string(inspect.getsource(priority9))
# %timeit is_function_safe_string(inspect.getsource(priority10))
# %timeit is_function_safe_string(inspect.getsource(priority11))

In [106]:
is_function_safe(priority9)

True

In [67]:
print(is_function_safe(priority1))
print(is_function_safe(priority2))
print(is_function_safe(priority3))
print(is_function_safe(priority4))
print(is_function_safe(priority5))
print(is_function_safe(priority6))
print(is_function_safe(priority7))
print(is_function_safe(priority8))
print(is_function_safe(priority9))
print(is_function_safe(priority10))
print(is_function_safe(priority11))

True
True
True
True
True
True
True
True
True
True
True


In [55]:
def priority_v2(v: tuple[int, ...], n: int) -> float:
  """A slightly modified version of ⁠ priority_v1 ⁠. This version penalizes vectors with repeated components."""
  penalty = 0.1 * sum(1 for i in set(v) if v.count(i) > 1)
  return penalty

In [56]:
is_function_safe(priority_v2)

True

In [57]:
def priority_v3(p: int, n: int) -> int:
  """
  Improved version of ⁠ priority_v0 ⁠.
  Ensures the returned residue class is not a factor of any element in a prime factorization of n.
  """
  potential_numbers = list(range(1, n))
  i = 2
  while i * i <= n:
    if n % i:
      i += 1
    else:
      while i in potential_numbers:
        potential_numbers.remove(i)
      n //= i
  if n > 1 and n in potential_numbers:
    potential_numbers.remove(n)
  if potential_numbers:
    return np.random.choice(potential_numbers)
  else:
    return np.random.choice(range(1, n))

In [58]:
is_function_safe(priority_v3)

True

In [59]:
def priority_v2(p: int, n: int) -> int:
  """
  Improved version of ⁠ priority_v0 ⁠.
  Ensures the returned residue class does not divide any prime factor of n.
  """
  assert n > 1, "n should be greater than 1"
  potential_numbers = list(range(1, n))
  for i in range(2, int(np.sqrt(n)) + 1):
    if n % i == 0:
      while i in potential_numbers:
        potential_numbers.remove(i)
      while n // i in potential_numbers:
        potential_numbers.remove(n // i)
  if potential_numbers:
    return np.random.choice(potential_numbers)
  else:
    return np.random.choice(range(1, n))

In [60]:
is_function_safe(priority_v2)

True

In [61]:
def priority_v2(v: tuple[int, ...], n: int) -> float:
  """Improved version of `priority_v1`.
  """
  if n < 3:
    raise ValueError("Vector length must be at least 3.")

  unique_pairs = len(set(itertools.combinations(v, 2)))
  max_pairs = (n * (n - 1) * (n - 2)) // 6

  # Calculate the number of independent subspaces
  # This is equivalent to the number of 2x2 sub-matrices with non-zero determinants
  independent_subspaces = sum(np.linalg.det(v[i:i+2].reshape((2, 2))) != 0 for i in range(0, n-1, 2))

  # Adjust the score to give more weight to higher unique pair counts and less dependency
  unique_pairs_adj = unique_pairs ** (1 / (n - 1))
  determinant_adj = 1 / (n - 1) ** (independent_subspaces)

  return unique_pairs_adj * determinant_adj / max_pairs

is_function_safe(priority_v2)

True

In [62]:
def priority_v3(v: tuple[int, ...], n: int) -> float:
  """Further improved version of `priority_v1`.
  """
  unique_counts = np.bincount(v, minlength=3)
  two_counts = np.bincount(unique_counts, minlength=2)
  three_counts = np.bincount(unique_counts, minlength=3)

  # Calculate the count of unique vectors and vectors with 2 or 3 occurrences
  unique_counts_nonzero = unique_counts[unique_counts > 0].sum()
  two_counts_nonzero = two_counts[two_counts > 0].sum()

  # Calculate the number of vectors with 3 occurrences
  three_counts_nonzero = three_counts[three_counts > 0].sum()

  # Penalty for vectors with 3 occurrences
  penalty = 3 * three_counts_nonzero

  # Penalty for vectors with 2 occurrences, considering their multiplicity
  two_counts_penalty = sum(i * (i - 1) for i in two_counts[two_counts > 0])

  # Return the priority value
  return np.prod(unique_counts) * (n - unique_counts_nonzero) * len(two_counts[two_counts > 0]) * np.exp(-penalty - 0.5 * two_counts_penalty)

is_function_safe(priority_v3)

True

In [63]:
def priority_v2(v: tuple[int, ...], n: int) -> float:
  """Improved version of `priority_v1`.
  """
  sums, freqs = set(), {}
  for _ in itertools.combinations(v, 3):
    sum_ = sum(x for x in _)
    sums.add(sum_)
    if sum_ not in freqs:
      freqs[sum_] = 0
    freqs[sum_] += 1
  return len(sums) / (n * (n - 1) * (n - 2)) + len(freqs)

is_function_safe(priority_v2)

True